# Video Games Review: Analysis

In this project, I explore [Metacritic's video games reviews](https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page=0).


## Setup

In this section, I import all necessary libraries, setup the SQLite database, and then import the input `.csv` files in a format that allows manipulation by both `sqlite` and `pandas`:


In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine

# creating SQLite database
engine = create_engine("sqlite://", echo=False)

In [3]:
from glob import glob

csv_paths = glob("./input/*.csv")

dfs = {}
for c in csv_paths:
    c_cleaned = c[8:].replace(".csv", "")
    dfs[c_cleaned] = pd.read_csv(c).sort_values(["name", "platform"])
    dfs[c_cleaned].to_sql(c_cleaned, engine, if_exists="replace", index=False)
    display(c_cleaned, dfs[c_cleaned].head(3))


'meta'

,meta_score,meta_rank,name,platform,release_date,summary
12533,69,12511.0,#DRIVE,Platform: Switch,"February 16, 2021",#DRIVE is an endless driving videogame inspire...
32163,69,12498.0,#DRIVE,Platform: Switch,"February 16, 2021",#DRIVE is an endless driving videogame inspire...
6709,77,6687.0,#IDARB,Platform: Xbox One,"January 30, 2015",#IDARB started as a picture of red box. Then m...


'user'

,user_score,user_rank,name,platform,release_date,summary
15251,6.8,11698.0,#DRIVE,Platform: Switch,"February 16, 2021",#DRIVE is an endless driving videogame inspire...
29149,6.8,11678.0,#DRIVE,Platform: Switch,"February 16, 2021",#DRIVE is an endless driving videogame inspire...
14945,6.9,11392.0,#IDARB,Platform: Xbox One,"January 30, 2015",#IDARB started as a picture of red box. Then m...


## Cleaning

### Joining two tables

In [4]:
# merging
merged_df = pd.read_sql_query(
    """
    SELECT m.meta_score, 
        u.user_score,
        u.name,
        u.platform,
        u.release_date,
        u.summary
    FROM user u
    LEFT JOIN meta m
        USING (name, platform, release_date) 
    UNION
    SELECT m.meta_score,
        u.user_score,
        m.name,
        m.platform,
        m.release_date,
        m.summary
    FROM meta m
        LEFT JOIN user u
            USING (name, platform, release_date)
    ORDER BY name, platform
    """,
    engine,
)

merged_df.head()


,meta_score,user_score,name,platform,release_date,summary
0,None,<<<<<<< HEAD,None,None,None,None
1,None,=======,None,None,None,None
2,None,>>>>>>> main,None,None,None,None
3,<<<<<<< HEAD,None,None,None,None,None
4,=======,None,None,None,None,None


### Ensuring Datetime data type

As SQLite does not support month names ([StackOverflow](https://stackoverflow.com/questions/1181123/date-formatting-from-sqlite-query)) as is found in the `release_date` column, we use [`pandas.to_datetime()`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) method instead:


In [5]:
# convert "release_date" column to datetime format
merged_df["release_date"] = pd.to_datetime(merged_df.release_date, format="%B %d, %Y")
merged_df.head()

,meta_score,user_score,name,platform,release_date,summary
0,None,<<<<<<< HEAD,None,None,NaT,None
1,None,=======,None,None,NaT,None
2,None,>>>>>>> main,None,None,NaT,None
3,<<<<<<< HEAD,None,None,None,NaT,None
4,=======,None,None,None,NaT,None


### Removing string prefix in `platform` column

As all values in the `platform` column has a `"Platform: "` prefix, I use SQL's `SUBSTR()` to begin the string at a later index.

If the the unwanted substring is not consistently a prefix, I would use `REPLACE()` instead.


In [6]:
# displaying distinct values of `platform` column to confirm all values start with prefix
merged_df.to_sql("games", engine, if_exists="replace", index=False)
pd.read_sql_query("SELECT DISTINCT platform FROM games", engine)


,platform
0,None
1,Platform: Switch
2,Platform: Xbox One
3,Platform: Xbox 360
4,Platform: PlayStation 3
5,Platform: PC
6,Platform: PlayStation 4
7,Platform: PlayStation 2
8,Platform: PlayStation
9,Platform: DS


In [7]:
merged_df = pd.read_sql_query(
    """
    SELECT name,
        SUBSTR(platform, 11) platform,
        release_date,
        summary,
        meta_score,
        user_score
    FROM 
        games
        """,
    engine,
)
merged_df.head()


,name,platform,release_date,summary,meta_score,user_score
0,None,None,None,None,None,<<<<<<< HEAD
1,None,None,None,None,None,=======
2,None,None,None,None,None,>>>>>>> main
3,None,None,None,None,<<<<<<< HEAD,None
4,None,None,None,None,=======,None


### Missing Values

Though there are missing values in the dataset, there is NFA to impute as:

1. The `meta_score` and `user_score` columns are missing if there are <4 and <7 reviews for that game respectively, and is not possible to impute.
2. The `summary` of the game is authored by Metacritic, which is also difficult to impute.


In [8]:
pd.read_sql_query(
    """
    SELECT SUM(CASE WHEN meta_score IS NULL THEN 1 ELSE 0 END) meta_score_miss,
        SUM(CASE WHEN user_score IS NULL THEN 1 ELSE 0 END) user_score_miss,
        SUM(CASE WHEN name IS NULL THEN 1 ELSE 0 END) name_miss,
        SUM(CASE WHEN platform IS NULL THEN 1 ELSE 0 END) platform_miss,
        SUM(CASE WHEN release_date IS NULL THEN 1 ELSE 0 END) release_date_miss,
        SUM(CASE WHEN summary IS NULL THEN 1 ELSE 0 END) summary_miss
    FROM games
    """,
    engine,
)


,meta_score_miss,user_score_miss,name_miss,platform_miss,release_date_miss,summary_miss
0,3,1416,6,6,6,100


### Duplicates


In [9]:
# deduplicate first as no entries should be identical for all columns
merged_df = merged_df.drop_duplicates()
merged_df.to_sql("games", engine, if_exists="replace", index=False)

# check duplicates for "name", "summary" columns, as the other attributes can be expected to have duplicates
# exclude null values using `WHERE` statement as these will duplicate
for col in ["name", "summary"]:
    dup_df = pd.read_sql_query(
        f"""
        SELECT {col}, COUNT(*) counts
        FROM games
        WHERE {col} NOT NULL
        GROUP BY 1
        HAVING counts > 1
        ORDER BY 2 DESC
        """,
        engine,
    )
    if not dup_df.empty:
        display(col, dup_df)


'name'

,name,counts
0,Madden NFL 07,9
1,TMNT,8
2,Marvel: Ultimate Alliance,8
3,Madden NFL 06,8
4,Grand Theft Auto V,8
...,...,...
4226,2014 FIFA World Cup Brazil,2
4227,1942: Joint Strike,2
4228,187 Ride or Die,2
4229,10 Second Ninja X,2


'summary'

,summary,counts
0,The game involves players creating and destroy...,7
1,Need for Speed ProStreet accelerates street ra...,7
2,Marvel: Ultimate Alliance is an action/RPG tha...,7
3,LEGO Indiana Jones: The Original Adventures ta...,7
4,"Iron Man, an explosive third-person action sho...",7
...,...,...
3131,'DARK SOULS II Crown of the Ivory King' is the...,2
3132,"""Street Fighter"" is best known for its well-po...",2
3133,"""Our goal was to create a game that is perfect...",2
3134,"""Only the possibility of you can change our fa...",2


The `name` column has duplicates largely due to each row representing a game for a particular _platform_. Eg: if a game was released on PC, PS4 and PS5 -- each of these will have its own entry.

However, if there are games that also duplicate when the `name` and `platform` columns are grouped, the entry that has incorrect attributes should be removed.

In [10]:
pd.read_sql_query(
    """
    SELECT name,
        platform,
        COUNT(*) counts
    FROM games
    GROUP BY 1, 2
    HAVING counts > 1
    ORDER BY 3 DESC
    """,
    engine,
)


,name,platform,counts
0,None,None,6
1,Lil Gator Game,Switch,4
2,Mount & Blade II: Bannerlord,PC,4
3,One Piece Odyssey,PlayStation 5,4
4,Potion Craft: Alchemist Simulator,PC,4
...,...,...,...
248,X-Men Legends II: Rise of Apocalypse,PSP,2
249,X-Men: Reign of Apocalypse,Game Boy Advance,2
250,Young Souls,Switch,2
251,Ys VIII: Lacrimosa of DANA,PlayStation 5,2


In [11]:
# find entries for above games (and their platform) which duplicates
# create "name_platform" helper column to identify unique name+platform pairs
dups_df = pd.read_sql_query(
    """
    WITH dups_name_platform AS (
        SELECT name, 
            platform,
            COUNT(*) counts
        FROM games
        GROUP BY 1, 2
        HAVING counts > 1
    )
    
    SELECT g.*,
        REPLACE(LOWER(name), ' ', '-') || '//' || REPLACE(LOWER(platform), ' ', '-') name_platform
    FROM games g
    JOIN dups_name_platform
    USING (name, platform)
    """,
    engine,
)

dups_df


,name,platform,release_date,summary,meta_score,user_score,name_platform
0,13 Sentinels: Aegis Rim,Switch,2022-04-12 00:00:00.000000,"Vanillaware, the storytellers behind Odin Sphe...",88,8.6,13-sentinels:-aegis-rim//switch
1,13 Sentinels: Aegis Rim,Switch,2022-04-12 00:00:00.000000,"Vanillaware, the storytellers behind Odin Sphe...",88,8.7,13-sentinels:-aegis-rim//switch
2,3000th Duel,Switch,2020-02-19 00:00:00.000000,"A hero with no memory, and with a mysterious m...",71,8.1,3000th-duel//switch
3,3000th Duel,Switch,2020-02-19 00:00:00.000000,"A hero with no memory, and with a mysterious m...",71,8.2,3000th-duel//switch
4,A Hat in Time,Switch,2019-10-18 00:00:00.000000,A Hat in Time is a 3D collect-a-thon platforme...,76,7.9,a-hat-in-time//switch
...,...,...,...,...,...,...,...
514,Young Souls,Switch,2022-03-10 00:00:00.000000,"As orphans, Jenn and Tristan's life path broug...",76,5.0,young-souls//switch
515,Ys VIII: Lacrimosa of DANA,PlayStation 5,2022-11-15 00:00:00.000000,Ys returns with a brand new adventure for the ...,82,8.0,ys-viii:-lacrimosa-of-dana//playstation-5
516,Ys VIII: Lacrimosa of DANA,PlayStation 5,2022-11-15 00:00:00.000000,Ys returns with a brand new adventure for the ...,83,8.0,ys-viii:-lacrimosa-of-dana//playstation-5
517,Yu-Gi-Oh! 5D's Stardust Accelerator: World Cha...,DS,2009-05-19 00:00:00.000000,In Yu-Gi-Oh! 5D's Stardust Accelerator: World ...,73,7.4,yu-gi-oh!-5d's-stardust-accelerator:-world-cha...


In [12]:
# confirm on site the actual score of the duplicated entry
from scraper import setup_chrome_driver
from selenium.webdriver.common.by import By

DRIVER = setup_chrome_driver()
if not dups_df.empty:
    for n in set(dups_df.name_platform):
        name_platform_split = n.split("//")
        game_name_cleaned = name_platform_split[0]
        platform_name_cleaned = name_platform_split[1]

        game_url = f"https://www.metacritic.com/game/{platform_name_cleaned}/{game_name_cleaned}"

        DRIVER.get(game_url)
        metascore_indiv = float(
            DRIVER.find_element(
                By.XPATH,
                '//*[@id="main"]/div/div[1]/div[1]/div[3]/div/div[2]/div[1]/div[1]/div/div/a/div',
            ).text
        )
        userscore_indiv = float(
            DRIVER.find_element(
                By.XPATH,
                '//*[@id="main"]/div/div[1]/div[1]/div[3]/div/div[2]/div[1]/div[2]/div[1]/div/a/div',
            ).text
        )
        DRIVER.close()

        # check that a row with those values exist
        check_entry_df = dups_df[
            (dups_df.name_platform == n)
            & (dups_df.meta_score == float(metascore_indiv))
            & (dups_df.user_score == float(userscore_indiv))
        ]
        if check_entry_df.empty:
            raise Exception(
                f"""The game "{check_entry_df.name.values[0]}" on "{check_entry_df.platform.values[0]}" has incorrect Metascore and/or User Score. Please re-run scraper.py or manually update the CSV(s)."""
            )

        # replace scores with correct ones and dedup
        game_name = check_entry_df.name.values[0]
        platform_name = check_entry_df.platform.values[0]

        merged_df.loc[
            (merged_df.name == game_name) & (merged_df.platform == platform_name),
            ["meta_score"],
        ] = metascore_indiv
        merged_df.loc[
            (merged_df.name == game_name) & (merged_df.platform == platform_name),
            ["user_score"],
        ] = userscore_indiv

        merged_df.drop_duplicates(inplace=True)

        display(merged_df[merged_df.name == check_entry_df.name.values[0]])


SyntaxError: invalid syntax (scraper.py, line 5)

Similarly, `summary` column duplicates due to each row referring to a game that, despite being released on different platforms, have the same game summary.

In [ ]:
merged_df.to_sql("games", engine, if_exists="replace", index=False)

pd.read_sql_query(
    """
    SELECT name,
        platform,
        summary,
        COUNT(*) counts
    FROM games
    WHERE summary NOT NULL
    GROUP BY 1, 2, 3
    ORDER BY 4 DESC
    """,
    engine,
)


## Analysis

### Metascore vs User Scores
There are up to two scores (assuming sufficient rating volumes) for each game: 
* A game's **Metascore** is the [weighted average of critics' and publications' reviews](https://www.metacritic.com/about-metascores).
* A game's **user score** is the mean of users' scores. However, this score has been criticised online as users can provide reviews despite not owning the game (*[Reddit](https://www.reddit.com/r/NintendoSwitch/comments/rjn72b/metacritic_criticuser_reviews_do_they_mean/)*).

In [ ]:
# calculate Pearson Correlation Coefficient
pd.read_sql_query("""
    SELECT AVG(
        (meta_score - avg_x) * (user_score - avg_y)
    ) * AVG (
        (meta_score - avg x) * (user_score - avg_y)
    ) / (var_x * var_y) AS R2
    FROM games,
        (
            SELECT avg_x,
                avg_y,
                AVG(
                    (meta_score - avg_x) * (meta_score - avg_x)
                ) AS var_x,
                AVG(
                    (user_score - avg_y) * (user_score - avg_y)
                ) AS var_y
                FROM games,
                    (
                        SELECT AVG(meta_score) AS avg_x,
                            AVG(user_score) AS avg_y
                        FROM games
                    )
        )
    """, engine)

In [ ]:
# what is the relationship between metascore and userscores?
import matplotlib.pyplot as plt
import seaborn as sns

# need to use some other technique to visualise better because too many!
sns.scatterplot(data=merged_df, x="user_score", y="meta_score")
plt.show()


In [ ]:
# any patterns about the outliers?

In [ ]:
# merge in genres of games and see if there's insights about certain genres performing better? what about release years, months, decades? what about platforms?

In [ ]:
# "Must Play" Award and other awards (see if you can extract from square brackets)